In [1]:
//George Jen, Jen Tek LLC
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
import org.apache.spark.util.LongAccumulator
import org.apache.log4j._
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql._

Intitializing Scala interpreter ...

Spark Web UI available at http://bigdata:4040
SparkContext available as 'sc' (version = 3.0.0-preview, master = local[*], app id = local-1577080719841)
SparkSession available as 'spark'


import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
import org.apache.spark.util.LongAccumulator
import org.apache.log4j._
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql._


In [2]:
Logger.getLogger("org").setLevel(Level.ERROR)
val spark = SparkSession
    .builder
    .appName("csv2parquet")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", "file:///d:/tmp")
    .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@44236390


In [3]:
val ds = spark.read.format("csv").option("header", "true").option("quote", "\"").load("D:/teaching/scala/ticker_symbol.csv")
val df: DataFrame = ds.toDF()
df.show(3, false)

+------+-----------------------------+--------+---------------------------+--------------+----+
|Ticker|Name                         |Exchange|CategoryName               |CategoryNumber|_c5 |
+------+-----------------------------+--------+---------------------------+--------------+----+
|AUB.AX|Austbrokers Holdings Limited |ASX     |Accident & Health Insurance|431           |null|
|GLRE  |Greenlight Capital Re, Ltd.  |NMS     |Accident & Health Insurance|431           |null|
|SFG   |StanCorp Financial Group Inc.|NYQ     |Accident & Health Insurance|431           |null|
+------+-----------------------------+--------+---------------------------+--------------+----+
only showing top 3 rows



ds: org.apache.spark.sql.DataFrame = [Ticker: string, Name: string ... 4 more fields]
df: org.apache.spark.sql.DataFrame = [Ticker: string, Name: string ... 4 more fields]


In [4]:
//When the CSV file was read into DataFrame, all fields are String, below is to cast it to
//what the data should be, such as cast CategoryNumber to Int

val df_with_datatype=df.selectExpr("Ticker",
                  "Name", 
                  "Exchange",
                  "CategoryName",
                  "cast(CategoryNumber as int) CategoryNumber")

df_with_datatype.show(3, false)

+------+-----------------------------+--------+---------------------------+--------------+
|Ticker|Name                         |Exchange|CategoryName               |CategoryNumber|
+------+-----------------------------+--------+---------------------------+--------------+
|AUB.AX|Austbrokers Holdings Limited |ASX     |Accident & Health Insurance|431           |
|GLRE  |Greenlight Capital Re, Ltd.  |NMS     |Accident & Health Insurance|431           |
|SFG   |StanCorp Financial Group Inc.|NYQ     |Accident & Health Insurance|431           |
+------+-----------------------------+--------+---------------------------+--------------+
only showing top 3 rows



df_with_datatype: org.apache.spark.sql.DataFrame = [Ticker: string, Name: string ... 3 more fields]


In [5]:
//Save the DataFrame to Parquet format, overwrite if existing.
//Parquet is Columnar, good for Analytics query.

df_with_datatype.write.mode(SaveMode.Overwrite).parquet("D:/teaching/scala/ticker_symbol.parquet")


In [6]:
//Read the Parquet data back and run SQL query on it

val read_parquet_df = spark.read.parquet("D:/teaching/scala/ticker_symbol.parquet")

read_parquet_df.show(3, false)

+------+-----------------------------+--------+---------------------------+--------------+
|Ticker|Name                         |Exchange|CategoryName               |CategoryNumber|
+------+-----------------------------+--------+---------------------------+--------------+
|AUB.AX|Austbrokers Holdings Limited |ASX     |Accident & Health Insurance|431           |
|GLRE  |Greenlight Capital Re, Ltd.  |NMS     |Accident & Health Insurance|431           |
|SFG   |StanCorp Financial Group Inc.|NYQ     |Accident & Health Insurance|431           |
+------+-----------------------------+--------+---------------------------+--------------+
only showing top 3 rows



read_parquet_df: org.apache.spark.sql.DataFrame = [Ticker: string, Name: string ... 3 more fields]


In [7]:
import spark.implicits._
val TickerSymbol = read_parquet_df.toDF()
TickerSymbol.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Exchange: string (nullable = true)
 |-- CategoryName: string (nullable = true)
 |-- CategoryNumber: integer (nullable = true)



import spark.implicits._
TickerSymbol: org.apache.spark.sql.DataFrame = [Ticker: string, Name: string ... 3 more fields]


In [8]:
TickerSymbol.createOrReplaceTempView("TickerSymbol")
spark.sql("SELECT * from TickerSymbol where Ticker in ('IBM','MSFT','HPQ','GE')").show(20,false)

+------+-------------------------------------------+--------+-------------------------------+--------------+
|Ticker|Name                                       |Exchange|CategoryName                   |CategoryNumber|
+------+-------------------------------------------+--------+-------------------------------+--------------+
|MSFT  |Microsoft Corporation                      |NMS     |Business Software & Services   |826           |
|HPQ   |Hewlett-Packard Company                    |NYQ     |Diversified Computer Systems   |810           |
|GE    |General Electric Company                   |NYQ     |Diversified Machinery          |622           |
|IBM   |International Business Machines Corporation|NYQ     |Information Technology Services|824           |
+------+-------------------------------------------+--------+-------------------------------+--------------+

